In [16]:
import os
from typing import *

# pytorch Hooks

https://medium.com/the-dl/how-to-use-pytorch-hooks-5041d777f904

In [17]:
class VerposeExcution(nn.Module):
    def __init__(
        self,
        model: nn.Module
    ) -> None:
        super(VerposeExcution,self).__init__()
        
        self.model = model
        
        for name, layer in self.model.named_children():
            layer.__name__ = name
            layer.register_forward_hook(
                lambda layer, _, output: print(f"{layer.__name__}: {output.shape}")
            )
            
    def forward(self, x: Tensor) -> Tensor:
        return self.model(x)

NameError: name 'nn' is not defined

In [18]:
from torchvision.models import resnet50

In [19]:
verpose = VerposeExcution(resnet50())
dummy = torch.ones(1,3,224,224)

_ = verpose(dummy)

NameError: name 'VerposeExcution' is not defined

In [ ]:

from typing import Dict, Iterable, Callable

class FeatureExtractor(nn.Module):
    def __init__(self, model: nn.Module, layers: Iterable[str]):
        super().__init__()
        self.model = model
        self.layers = layers
        self._features = {layer: torch.empty(0) for layer in layers}

        for layer_id in layers:
            layer = dict([*self.model.named_modules()])[layer_id]
            layer.register_forward_hook(self.save_outputs_hook(layer_id))

    def save_outputs_hook(self, layer_id: str) -> Callable:
        def fn(_, __, output):
            self._features[layer_id] = output
        return fn

    def forward(self, x: Tensor) -> Dict[str, Tensor]:
        _ = self.model(x)
        return self._features


In [ ]:
resnet_features = FeatureExtractor(resnet50(), layers=["layer4", "avgpool"])
features = resnet_features(dummy)

In [ ]:
print({name: output.shape for name, output in features.items()})


In [ ]:
x = torch.empty(0)
x =  

# Python decorators

https://realpython.com/primer-on-python-decorators/#returning-functions-from-functions

In [20]:
def add_one(number: int):
    return number + 1

add_one(2)

3

## function behaviour

In [21]:
def say_hello(name: str) -> str:
    return f'hello {name}'

def be_awesome(name: str) -> str:
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func: Callable) -> str:
    return greeter_func('Bob')

In [22]:
greet_bob(say_hello)

'hello Bob'

## inner Function

In [23]:
def parent() -> None:
    print("Printing from the parent() function")
    
    def first_child() -> None:
        print("Printing from the _child() function")
    
    def second_child() -> None:
        print("Printing from the second_child() function")

    second_child()        
    first_child()


In [24]:
parent()

Printing from the parent() function
Printing from the second_child() function
Printing from the _child() function


In [25]:
first_child()

NameError: name 'first_child' is not defined

## Returning Functions From Functions

In [26]:
def parent(num: int) -> str:
    
    def first_child() -> str:
        return 'Hi, I am lima'
    
    def second_child() -> str:
        return 'call me lima'
    
    if num == 1:
        return first_child
    
    elif num == 2:
        return second_child
    
    else:
        return 'invalid input'

In [27]:
first =  parent(1)
first()

'Hi, I am lima'

## simple decorators

In [28]:
def my_decorator(func: Callable) -> Callable:
    
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
        
    return wrapper
        
def say_hello() -> str:
    print('hello')

say_hello = my_decorator(say_hello)
say_hello()

Something is happening before the function is called.
hello
Something is happening after the function is called.


In [29]:
say_hello = my_decorator(say_hello)
say_hello()

Something is happening before the function is called.
Something is happening before the function is called.
hello
Something is happening after the function is called.
Something is happening after the function is called.


In [30]:
from datetime import datetime

def not_during_night(func: Callable) -> Callable or str:
    
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            print('not allowed')
            
    return wrapper


@not_during_night
def say_whee():
    print('Whee')
    


In [31]:
say_whee()

not allowed


In [32]:
datetime.now().hour

5

## Syntactic Sugar!

In [33]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

In [34]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [35]:
from datetime import datetime

def not_during_night(func: Callable) -> Callable or str:
    
    def wrapper():
        if 7 <= datetime.now().hour < 11:
            func()
        else:
            print('not allowed')
            
    return wrapper


@not_during_night
def say_whee():
    print('Whee')

In [36]:
say_whee()

not allowed


## Reusing Decorators

In [37]:
def do_twice(func: Callable) -> Callable:
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

In [38]:
@do_twice
def say_whee() -> None:
    print('Whee!')

In [39]:
say_whee()

Whee!
Whee!


## Decorating Functions With Arguments

In [40]:
@do_twice
def greet(name: str) -> None:
    print(f'hell {name}')

In [41]:
greet('saeed')

TypeError: wrapper_do_twice() takes 0 positional arguments but 1 was given

In [42]:
def do_twice(func: Callable) -> Callable:
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

In [43]:
@do_twice
def greet(name: str) -> None:
    print(f'hello {name}')
    
greet('saeed')

hello saeed
hello saeed


In [44]:
@do_twice
def say_whee() -> None:
    print('Whee!')
say_whee()

Whee!
Whee!


In [45]:
@do_twice
def return_greeting(name: str) -> str:
    print("Creating greeting")
    return f'hi {name}'

In [46]:
x = return_greeting('saeed')


Creating greeting
Creating greeting


In [47]:
def do_twice(func: Callable) -> Callable:
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [48]:
@do_twice
def return_greeting(name: str) -> str:
    print("Creating greeting")
    return f'hi {name}'

In [49]:
x = return_greeting('saeed')
print(x)

Creating greeting
Creating greeting
hi saeed


## Who Are You, Really?

In [50]:
print

<function print>

In [51]:
print.__name__

'print'

In [52]:
say_hello.__name__

'wrapper'

In [53]:
help(say_whee)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice(*args, **kwargs)



In [54]:
import functools


In [55]:
def do_twice(func):
    @functoolst.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

In [56]:
say_whee

<function __main__.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

## decorator template

In [57]:
def decorator(func: Callable) -> Callable:
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

## A Few Real World Examples

In [58]:
import time

In [59]:
def timer(func: Callable):
    
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time -start_time
        print(f"Finished {func.__name__!r} in {total_time:.4f} secs")
        return value
    return wrapper_timer


In [60]:
@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i for i in range(10000)])


In [99]:
x = waste_some_time(2201)

Finished 'waste_some_time' in 0.8898 secs


In [62]:
print(x)

None


In [63]:
def debug(func: Callable) -> Callable:
    
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr) 
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}") 
        return value
    return wrapper_debug

In [64]:
@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"

In [65]:
x = make_greeting('saeed', age=25)

Calling make_greeting('saeed', age=25)
'make_greeting' returned 'Whoa saeed! 25 already, you are growing up!'


In [66]:
import math

In [67]:
@debug
def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

In [68]:
approximate_e(100)

Calling approximate_e(100)
'approximate_e' returned 2.7182818284590455


2.7182818284590455

In [69]:
def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down


In [70]:
@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [71]:
countdown(3)

3
2
1
Liftoff!


In [72]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [73]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [74]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'class Circle:\n    def __init__(self, radius):\n        self._radius = radius\n\n    @property\n    def radius(self):\n        """Get value of radius"""\n        return self._radius\n\n    @radius.setter\n    def radius(self, value):\n        """Set radius, raise error if negative"""\n        if value >= 0:\n            self._radius = value\n        else:\n            raise ValueError("Radius must be positive")\n\n    @property\n    def area(self):\n        """Calculate area inside circle"""\n        return self.pi() * self.radius**2\n\n    def cylinder_volume(self, height):\n        """Calculate volume of cylinder with circle as base"""\n        return self.area * height\n\n    @classmethod\n    def unit_circl

In [75]:
def det_tyep(func):
    
    @functools.wraps(func)
    def wrapper_type(*args, **kwargs):
        
        value = func(*args, **kwargs)
        
        return f'this input of type {value}'
    
    return wrapper_type

In [76]:
@det_tyep
def return_tyep(inputx):
    return type(inputx)
    
return_tyep([5])    
    

"this input of type <class 'list'>"

In [77]:
type(5)

int

In [78]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [79]:
x = Circle.unit_circle()

In [80]:
Circle.unit_circle()

In [81]:
x.setter(2)

AttributeError: 'Circle' object has no attribute 'setter'

In [83]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num
    @timer
    def waste_time(self,num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [85]:
tw = TimeWaster(100)

Calling __init__(<__main__.TimeWaster object at 0x7f3c117d37c0>, 100)
'__init__' returned None


In [86]:
tw.waste_time(999)

Finished 'waste_time' in 0.0420 secs


In [87]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

In [90]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [91]:
tw = TimeWaster(1000)

Finished 'TimeWaster' in 0.0000 secs


In [92]:
tw.waste_time(999)

In [112]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

In [113]:
@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

In [114]:
 greet('saeed')

Hello saeed
Hello saeed
Hello saeed
Hello saeed


In [100]:
def repeat(4):
    def decorator_repeat(greet):
        @functools.wraps(greet)
        def wrapper_repeat(name):
            for _ in range(4):
                value = greet(name)
            return value
        return wrapper_repeat
    return decorator_repeat

SyntaxError: invalid syntax (<ipython-input-100-363c6d1d391e>, line 1)

In [209]:
class Square_Area:
    def __init__(self, area_f):
        functools.update_wrapper(self, area_f)
        self.area_f = area_f

    
    def __call__(self,**kwargs):
        print(f'square areas is {self.area_f(kwargs)}')

In [210]:
@Square_Area
def sq_area(length = 1):
    return length**2

In [213]:
sq_area(length=5)

TypeError: unsupported operand type(s) for ** or pow(): 'dict' and 'int'

In [165]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)

In [173]:
@CountCalls
def say_whee():
    print("Whee!")

In [177]:
count =  CountCalls(say_whee)

In [186]:
count()

Call 8 of 'say_whee'
Whee!
